In [1]:
from transformers import pipeline
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import json

In [3]:
# Load the pre-trained emotion analysis pipeline
emotion_pipeline = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

Device set to use mps:0


In [4]:
with open("final_campaign_data.txt", "r") as file:
    data = json.load(file)

In [5]:
# Convert the JSON data to a Pandas DataFrame
campaigns = pd.DataFrame(data)

In [17]:
emotionally_charged_threshold = 0.75
def classify_emotion(description):
    try:
        result = emotion_pipeline(description)
        dominant_emotion = max(result, key=lambda x: x['score'])
        is_emotionally_charged = (
            dominant_emotion['score'] >= emotionally_charged_threshold and dominant_emotion['label'] != "neutral"
        )
        return dominant_emotion['label'], dominant_emotion['score'], is_emotionally_charged
    except Exception as e:
        return "Error", 0.0, False

In [23]:
import warnings
warnings.filterwarnings("ignore")

# Perform emotion analysis on all campaign title
campaigns1 = campaigns.head(20)
campaigns1[["DominantEmotion", "EmotionScore", "EmotionallyCharged"]] = campaigns1["Title"].apply(
    lambda desc: pd.Series(classify_emotion(desc))
)
campaigns1

,_id,URL,Category,image,GoalAmount,Organizer,Description,Updates,Comments,Created,Data_created_on,success,Title,RaisedAmount,DominantEmotion,EmotionScore,EmotionallyCharged
0,{'$oid': '61e5fb7e1ad7d6fcca9ad8ac'},https://www.gofundme.com/f/1-dollar-to-1-milli...,Community & Neighbors,"background-image: url(""https://images.gofundme...",1000000,"{'Organizer_name': 'Courteney Ridgeway', 'Orga...",Courteney Ridgeway is organizing this fundrais...,[],"[{'donated_amount': 20, 'comment': 'I believe ...","Created November 10, 2021",{'$date': {'$numberLong': '1642444078735'}},0,1 dollar to 1 million empowerment or experiment..,383.0,neutral,0.776969,False
1,{'$oid': '61e764a53d2cac2f33b659f6'},https://www.gofundme.com/f/1-girl-1-calculator,Education & Learning,"background-image: url(""https://images.gofundme...",2600,"{'Organizer_name': 'Macy Hill', 'Organizer_des...",Macy Hill is organizing this fundraiser on beh...,"[December 5, 2021, by Christine Hill, Benefici...","[{'donated_amount': 400, 'comment': 'As part o...","Created September 6, 2021",{'$date': {'$numberLong': '1642536533312'}},0,1 Girl 1 Calculator,1680.0,neutral,0.886611,False
2,{'$oid': '61dfa752697f4b953f55a291'},https://www.gofundme.com/f/1-year-of-event-pro...,"Creative Arts, Music & Film","background-image: url(""https://images.gofundme...",3500,"{'Organizer_name': 'Herine Palacios', 'Organiz...",Herine Palacios is organizing this fundraiser....,[],"[{'donated_amount': 5, 'comment': 'lets gooooo...","Created November 4, 2021",{'$date': {'$numberLong': '1642029314823'}},0,1 year of event programming,1120.0,neutral,0.510796,False
3,{'$oid': '61e206bb1ad7d6fcca9ac17b'},https://www.gofundme.com/f/10-secrets-to-stopp...,Environment,"background-image: url(""https://images.gofundme...",277477,"{'Organizer_name': 'Richard Eckstein', 'Organi...",Richard Eckstein is organizing this fundraiser...,[],[],"Created September 9, 2021",{'$date': {'$numberLong': '1642184811786'}},0,Secrets to Stopping Climate Change,10.0,neutral,0.698540,False
4,{'$oid': '61e637933d2cac2f33b654a2'},https://www.gofundme.com/f/10-thursday-with-gpbr,Non-Profits & Charities,"background-image: url(""https://images.gofundme...",1000,"{'Organizer_name': 'Heather Horlor', 'Organize...",Heather Horlor is organizing this fundraiser t...,[],[],"Created November 8, 2021",{'$date': {'$numberLong': '1642459459232'}},0,$10 Thursday with GPBR!,260.0,surprise,0.558378,False
5,{'$oid': '61e384e63d2cac2f33b648b7'},https://www.gofundme.com/f/10-year-old-in-a-co...,Accidents & Emergencies,"background-image: url(""https://images.gofundme...",150000,"{'Organizer_name': 'Tamara Gabbard', 'Organize...",Tamara Gabbard is organizing this fundraiser.\...,"[December 24, 2021, by Tamara Gabbard, Organiz...","[{'donated_amount': 50, 'comment': 'I was Insp...","Created November 16, 2021",{'$date': {'$numberLong': '1642282646338'}},0,10 Year Old In a Coma After Accident~HELP NEEDED,13410.0,neutral,0.332123,False
6,{'$oid': '61e642861ad7d6fcca9ada1d'},https://www.gofundme.com/f/100-blankets-for-10...,Community & Neighbors,"background-image: url(""https://images.gofundme...",1500,"{'Organizer_name': 'Lexi Berzok', 'Organizer_d...",Team fundraiser\nLexi Berzok and Jake Gordon a...,[],"[{'donated_amount': 10, 'comment': 'i wanna he...","Created November 12, 2021",{'$date': {'$numberLong': '1642462262402'}},0,100 blankets for 100 people in need,125.0,neutral,0.662018,False
7,{'$oid': '61e7a0f03d2cac2f33b65b28'},https://www.gofundme.com/f/100-from-100-challenge,Education & Learning,"background-image: url(""https://images.gofundme...",5000,{'Organizer_name': 'Billinghurst Middle School...,Billinghurst Middle School is organizing this ...,[],"[{'donated_amount': 100, 'comment': 'My sons a...","Created October 19, 2021",{'$date': {'$numberLong': '1642551968839'}},0,$100 from 100 Challenge,2340.0,neutral,0.671231,False
8,{'$oid': '61e4c7ec1ad7d6fcca9ad2c6'},https://www.gofundme.com/f/100-hill-bear-crawl...,Community & Neighbors,"background-ima

In [10]:
campaigns1

,_id,URL,Category,image,GoalAmount,Organizer,Description,Updates,Comments,Created,Data_created_on,success,Title,RaisedAmount,DominantEmotion
0,{'$oid': '61e5fb7e1ad7d6fcca9ad8ac'},https://www.gofundme.com/f/1-dollar-to-1-milli...,Community & Neighbors,"background-image: url(""https://images.gofundme...",1000000,"{'Organizer_name': 'Courteney Ridgeway', 'Orga...",Courteney Ridgeway is organizing this fundrais...,[],"[{'donated_amount': 20, 'comment': 'I believe ...","Created November 10, 2021",{'$date': {'$numberLong': '1642444078735'}},0,1 dollar to 1 million empowerment or experiment..,383.0,neutral
1,{'$oid': '61e764a53d2cac2f33b659f6'},https://www.gofundme.com/f/1-girl-1-calculator,Education & Learning,"background-image: url(""https://images.gofundme...",2600,"{'Organizer_name': 'Macy Hill', 'Organizer_des...",Macy Hill is organizing this fundraiser on beh...,"[December 5, 2021, by Christine Hill, Benefici...","[{'donated_amount': 400, 'comment': 'As part o...","Created September 6, 2021",{'$date': {'$numberLong': '1642536533312'}},0,1 Girl 1 Calculator,1680.0,neutral
2,{'$oid': '61dfa752697f4b953f55a291'},https://www.gofundme.com/f/1-year-of-event-pro...,"Creative Arts, Music & Film","background-image: url(""https://images.gofundme...",3500,"{'Organizer_name': 'Herine Palacios', 'Organiz...",Herine Palacios is organizing this fundraiser....,[],"[{'donated_amount': 5, 'comment': 'lets gooooo...","Created November 4, 2021",{'$date': {'$numberLong': '1642029314823'}},0,1 year of event programming,1120.0,neutral
